In [1]:
from googleapiclient.discovery import build
import pandas as pd
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()
api_key = os.getenv('YT_API_KEY')

### Find trending videos per query

In [3]:
req_json = build('youtube', 'v3', developerKey=api_key).search().list(
            part='snippet',
            maxResults=50,
            q='fishing',
            order='viewCount',
            type='video',
            regionCode='US',
            videoDuration='medium',
            publishedAfter='2023-01-01T00:00:00Z'
        ).execute()

In [4]:
user, user_id, vid_id, title, date = [], [], [], [], []

for item in req_json['items']:
    user.append(item['snippet']['channelTitle'])
    user_id.append(item['snippet']['channelId'])
    vid_id.append(item['id']['videoId'])
    title.append(item['snippet']['title'])
    date.append(item['snippet']['publishTime'])

df_temp = pd.DataFrame({
    'username': user,
    'user_id':user_id,
    'vid_id': vid_id,
    'date': date,
    'title': title
})

### Find subscribers per channel

In [6]:
user_id_out = ""
for i in set(user_id):
    user_id_out += i + ','
user_id_out = user_id_out[:-1]

In [10]:
subs_json = build('youtube', 'v3', developerKey=api_key).channels().list(
            part='statistics',
            id=user_id_out
        ).execute()

In [17]:
subs = {}
for i in subs_json['items']:
    channel_id = i['id']
    subs[channel_id] = i['statistics']['subscriberCount']

In [36]:
df_subs = pd.DataFrame(subs.items(), columns=['user_id','subscribers'])
df_subs['subscribers'] = df_subs['subscribers'].astype(int)

### Find views per video

In [21]:
vid_id_out = ""
for val in set(vid_id):
    vid_id_out += val + ','
vid_id_out = vid_id_out[:-1]

In [24]:
views_json = build('youtube', 'v3', developerKey=api_key).videos().list(
            part='statistics',
            id=vid_id_out
        ).execute()

In [26]:
views = {}
for i in views_json['items']:
    vid_id = i['id']
    views[vid_id] = i['statistics']['viewCount']

In [35]:
df_views = pd.DataFrame(views.items(), columns=['vid_id','views'])
df_views['views'] = df_views['views'].astype(int)

### Merge

In [41]:
df = df_temp.merge(df_subs, how='left', on='user_id')
df = df.merge(df_views, how='left', on='vid_id')
df['viewsToSubs'] = df['views']/df['subscribers']

### Which videos received the most views per subscriber?

In [46]:
# These videos found a way to become discoverable by new audiences
df.sort_values(by='viewsToSubs', ascending=False)

,username,user_id,vid_id,date,title,subscribers,views,viewsToSubs
48,Mim Fishing BD,UCQdAc7px0yik6OO9yyKT4LQ,d16068DfkFg,2023-02-06T12:47:34Z,Awesome Traditional Fishing Festival Of Villag...,5990,2323115,387.832220
24,Popular Fish Cutting,UCWkRq7mbzU9u4sFc7wTysNA,Gfx5sR1IBzk,2023-03-19T12:00:10Z,😱😱😱 I Never Seen || Giant Carp Fish Cutting Li...,19300,4295714,222.575855
36,Lady Hook Fishing,UCRDps63nt5yB4BRztGVUj9Q,EEPEfenpewA,2023-07-15T15:02:47Z,Fishing video || three lady catching big fish ...,19200,3119575,162.477865
25,The Lawless Tide,UCqD-eS-qRKsGi9LfT-4jiJA,RpNfMFNz2VM,2023-10-18T20:55:00Z,Someone Called the GAME WARDEN Because I Was C...,32200,4273708,132.723851
6,Pearl Girl,UCFruKhxhHaAd4EWjavRuKOA,NbXSj2WT6nw,2023-06-26T12:30:12Z,I awakened the pearl clam that had been sleepi...,201000,16419059,81.686861
4,Nana Fishing,UCFnlBSh16mfwts4iAM4wfFw,diA8Uoo2604,2023-03-30T11:30:10Z,"Fishing Videos: Survival Skills, Catch A Lot O...",282000,18858406,66.873780
45,Anuak fishing,UCZreNI7rMRPajS9Dfv3sGYA,IwnPFOqzwQ8,2023-04-10T11:20:00Z,menembak sidat terbesar di taiwan||langsung di...,41600,2381131,57.238726
44,Raww,UC9L9DwN8juHuiv9tlIurr3g,edKGqJhmJvM,2023-08-27T13:00:34Z,SNAPPING TURTLE ATTACKS ME,54300,2372027,43.683738
7,Fishing TV,UCc7Ml2Cugne27DBGkqLJurg,FQTQD3WHlCk,2023-04-24T02:22:45Z,Catch 100 Extremely Poisonous Black Gold Snake...,343000,14368221,41.889857
12,FISHGUM,UCwutwGfHyLloVtWwn19zS6g,XpcLvyjF7Hg,2023-06-01T19:06:04Z,I Tossed! A Fishing Camera Into The Surf And S...,188000,7141129,37.984729
